In [1]:
from modelsandbox import Model

# WITH APPROACH
model = Model()

# SPF Calculations
with model.root.add_layer() as layer:
    layer.add_schema({
        "label": "schema1",
        "params": ["x"],
        "actions": ["get"],
        "data": {
            0: 0,
            1: 10,
            2: 20
        }
    })

    # SPF 1
    with layer.add_sequence() as seq:
        seq.add_schema({
            "label": "schema2",
            "params": ["x"],
            "actions": ["get"],
            "data": {
                0: {"a": 0, "b": 0, "c": 0, "d": 0},
                1: {"a": 10, "b": 10, "c": 10, "d": 10},
                2: {"a": 20, "b": 20, "c": 20, "d": 20}
            }
        }, hidden=True)
        @seq.add_wrapped()
        def process(a, b, c, d):
            return a * b + c / d
        
    # SPF 2
    with layer.add_sequence() as seq:
        seq.add_schema({
            "label": "schema3",
            "params": ["x"],
            "actions": ["get"],
            "data": {
                0: {"a": 0, "b": 0, "c": 0, "d": 0},
                1: {"a": 10, "b": 10, "c": 10, "d": 10},
                2: {"a": 20, "b": 20, "c": 20, "d": 20}
            }
        }, hidden=True)
        @seq.add_wrapped()
        def process(a, b, c, d):
            return a * b + c / d

# AF Calculations
with model.root.add_layer() as layer:

    # AF 1
    layer.add_wrapped()
    def af_lane_width(lane_width):
        return lane_width / 12

In [1]:
class Test(object):

    def __init__(self) -> None:
        self.data = [0, 1, [4,5]]

    def __getitem__(self, key):
        return self.data[key]

In [3]:
Test()[[0,1]]

TypeError: list indices must be integers or slices, not list

In [2]:
model.analyze(x=1)

{'x': 1, 'schema1': 10, 'process': 101.0}

In [ ]:
model.root.hidden

In [ ]:
import importlib
import modelsandbox
importlib.reload(modelsandbox)
from modelsandbox import Model

In [ ]:
from modelsandbox import Model

model = Model()

model[0].set_label('First Layer')
model[0].set_tags('layer1')
model[0].add_function(lambda a, b: a + b, tags='tag1')
model[0].add_sequence()
model[0][1]

# Cost example

In [ ]:
import importlib
import modelsandbox
importlib.reload(modelsandbox)
from modelsandbox import Model

# Initialize the model class
model = Model()

# Add a layer to the model to compute airline ticket cost
model[0].set_label('Compute ticket cost')

# Add the process schema to the model
model[0].add_schema({
    "label": "ticket_cost",
    "params": ["destination", "airline_class"],
    "actions": ["get", "get"],
    "data": {
        "Chicago": {
            "Economy": 220,
            "Business": 450,
            "First": 785
        },
        "Los Angeles": {
            "Economy": 365,
            "Business": 520,
            "First": 965
        }
    }
})

# Add a layer to the model to compute additional costs
model[1].set_label('Compute expenses')

# Add a processor to compute travel cost
@model[1].add_wrapped(tags=['__cost'])
def travel_cost(number_of_travelers, ticket_cost):
    """
    Compute total travel cost for all travelers.
    """
    return number_of_travelers * ticket_cost

# Add processor to compute lodging cost
@model[1].add_wrapped(tags=['__cost'])
def lodging_cost(number_of_travelers, nightly_cost, number_of_nights):
    """
    Compute total lodging cost for all travelers.
    """
    return number_of_travelers * nightly_cost * number_of_nights

# Add processor to compute per diem
@model[1].add_wrapped(tags=['__cost'])
def per_diem_cost(number_of_travelers, number_of_nights, per_diem):
    """
    Compute total per diem cost for all travelers.
    """
    return number_of_travelers * number_of_nights * per_diem

# Add a layer to the model to aggregate costs
model[2].set_label('Aggregate expenses')

# Add processor to compute total trip cost
@model[2].add_wrapped()
def total_trip_cost(travel_cost, lodging_cost, per_diem_cost, __cost):
    """
    Compute total trip cost for all travelers.
    """
    return travel_cost + lodging_cost + per_diem_cost

model.structure

In [ ]:
model.tagged

In [ ]:
model.analyze(
    airline_class="Business",
    destination="Chicago",
    nightly_cost=185,
    number_of_nights=4,
    number_of_travelers=3,
    per_diem=72,
)

# Scratch

In [ ]:
from example import model

In [ ]:
model.analyze(
    airline_class="Business",
    destination="Chicago",
    nightly_cost=185,
    number_of_nights=4,
    number_of_travelers=3,
    per_diem=72,
    ticket_cost=500
)

In [ ]:
model.parameters

In [ ]:
from sample_model import model

In [ ]:
model.analyze(
    aadt = 580,
    curve_length = 0.2,
    curve_radius = 800,
    lane_width = 11,
    length = 0.6,
    observed_kabco = 3.4,
    shoulder_type = 'gravel',
    shoulder_width = 4,
    spiral = 'both'
)